In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Ingest drivers.json file

##### Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType

name_schema = StructType(fields = [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

drivers_schema = StructType(fields = [
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("name", name_schema),
    StructField("dob", DateType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

In [0]:
drivers_df = spark.read.schema(drivers_schema).json(f"{raw_folder_path}/drivers.json")
display(drivers_df)

##### Rename columns and add new columns

In [0]:
from pyspark.sql.functions import col, concat, lit

drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .drop(col("url"))
drivers_with_columns_df = add_ingestion_date(drivers_with_columns_df)
display(drivers_with_columns_df)

In [0]:
drivers_with_columns_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/drivers"))

In [0]:
dbutils.notebook.exit("Success")